# CS295B F19: Homework 1
## Anonymization, k-Anonymity, and the Laplace Mechanism
### Curtis Wilcox - September 17, 2019

## Instructions

The first half of this notebook contains code to read in and preprocess the example dataset. The second half contains questions for you to answer by writing code and describing your solutions.

First, download the example dataset and ensure that all cells in this notebook execute without error. If you have trouble getting the notebook to run, please post a question on Piazza. 

The point value of each question is listed with the question, and these add up to 100 points. The assignment is due by 5:00pm on Friday, September 13. When you have finished your assignment, submit it via Gradescope under the assignment "Homework 1." For questions on grading and submitting assignments, refer to the course webpage or email the instructor.

## Preamble: Read in Adult dataset & Preprocessing

Download the dataset by clicking [here](https://jnear.github.io/cs295-data-privacy/homework/adult_with_pii.csv) and placing it in the same directory as this notebook.

The dataset is based on census data. I have added the columns `Name`, `DOB`, `SSN`, and `Zip` to represent personally identifiable information (PII). The values in these columns are made up.

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import pandas as pd
import numpy as np

def your_code_here():
    return 1

adult_data = pd.read_csv("adult_with_pii.csv")
adult_data.head()

Name        DOB          SSN    Zip  Age         Workclass  \
0  Karrie Trusslove   9/7/1967  732-14-6110  64152   39         State-gov   
1  Brandise Tripony   6/7/1988  150-19-2766  61523   50  Self-emp-not-inc   
2     Brenn McNeely   8/6/1991  725-59-9860  95668   38           Private   
3       Dorry Poter   4/6/2009  659-57-4974  25503   53           Private   
4       Dick Honnan  9/16/1951  220-93-3811  75387   28           Private   

   fnlwgt  Education  Education-Num      Martial Status         Occupation  \
0   77516  Bachelors             13       Never-married       Adm-clerical   
1   83311  Bachelors             13  Married-civ-spouse    Exec-managerial   
2  215646    HS-grad              9            Divorced  Handlers-cleaners   
3  234721       11th              7  Married-civ-spouse  Handlers-cleaners   
4  338409  Bachelors             13  Married-civ-spouse     Prof-specialty   

    Relationship   Race     Sex  Capital Gain  Capital Loss  Hours per week  \
0  Not-in-family  White    Male          2174             0              40   
1        Husband  White    Male             0             0              13   
2  Not-in-family  White    Male             0             0              40   
3        Husband  Black    Male             0             0              40   
4           Wife  Black  Female             0             0              40   

         Country Target  
0  United-States  <=50K  
1  United-States  <=50K  
2  United-States  <=50K  
3  United-States  <=50K  
4           Cuba  <=50K

In [6]:
# Remove PII
adult_anon = adult_data.drop(columns=['Name', 'SSN'])
adult_anon.head()

DOB    Zip  Age         Workclass  fnlwgt  Education  Education-Num  \
0   9/7/1967  64152   39         State-gov   77516  Bachelors             13   
1   6/7/1988  61523   50  Self-emp-not-inc   83311  Bachelors             13   
2   8/6/1991  95668   38           Private  215646    HS-grad              9   
3   4/6/2009  25503   53           Private  234721       11th              7   
4  9/16/1951  75387   28           Private  338409  Bachelors             13   

       Martial Status         Occupation   Relationship   Race     Sex  \
0       Never-married       Adm-clerical  Not-in-family  White    Male   
1  Married-civ-spouse    Exec-managerial        Husband  White    Male   
2            Divorced  Handlers-cleaners  Not-in-family  White    Male   
3  Married-civ-spouse  Handlers-cleaners        Husband  Black    Male   
4  Married-civ-spouse     Prof-specialty           Wife  Black  Female   

   Capital Gain  Capital Loss  Hours per week        Country Target  
0          2174             0              40  United-States  <=50K  
1             0             0              13  United-States  <=50K  
2             0             0              40  United-States  <=50K  
3             0             0              40  United-States  <=50K  
4             0             0              40           Cuba  <=50K

In [4]:
# PII only
pii = adult_data[['Name', 'DOB', 'SSN', 'Zip']]
pii.head()

Name        DOB          SSN    Zip
0  Karrie Trusslove   9/7/1967  732-14-6110  64152
1  Brandise Tripony   6/7/1988  150-19-2766  61523
2     Brenn McNeely   8/6/1991  725-59-9860  95668
3       Dorry Poter   4/6/2009  659-57-4974  25503
4       Dick Honnan  9/16/1951  220-93-3811  75387

## END PREAMBLE
-------------

## Collaboration Statement

In the cell below, write your collaboration statement. This statement should describe all collaborations, even high-level ones (e.g. "I discussed my general approach for answering question 3 with Josh"). High-level collaborations of this kind are allowed as long as they are described; copying of answers or code is not allowed.

In [6]:
# In this cell (in markdown or a comment), write your collaboration statement

# I collaborated on a high-level basis with Jacob W.

## Question 1

(20 points)

Using the dataframes `pii` and `adult_anon`, perform a linking attack to recover the names of as many samples in `adult_anon` as possible.

How many names are you able to recover?

In [10]:
# In this cell, write code to perform the linking attack
    
# print(pii.head())
# adult_anon.head()  
attack = pd.merge(pii, adult_anon, left_on=['DOB', 'Zip'], right_on=['DOB', 'Zip'])

In [13]:
# In this cell, write code to determine how many names could be recovered

attack['Name'].value_counts().head()

# all but two names successfully recovered (uniquely)

Antonin Chittem    2
Barnabe Haime      2
Siffre Espinosa    1
Vance Wale         1
Janet Mitrikhin    1
Name: Name, dtype: int64

## Question 2

(30 points)

Implement a function `is_k_anonymous` to check (for a given `k`) whether a given dataframe satisfies k-Anonymity.

In [20]:
# In this cell, write code to implement 'is_k_anonymous'

def is_k_anonymous(k, iqs, df): # what is iqs? Unused in the function & not in the notes from class (this function is from notes with slight changes)
    for index, row in df.iterrows():
        query = ' & '.join([f'{col} == {row[col]}' for col in df.columns])
        rows = df.query(query)
        if rows.shape[0] < k:
            return False
    return True

## Question 3

(5 points)

In one or two sentences, informally describe how well you expect your implementation of 'is_k_anonymous' to scale with the size of the input data.

In [64]:
# In this cell, describe (in markdown or in a comment) the scaling behavior of your answer in question 2.

"""
The scaling behavior would not go well. It will always loop and check everything, even if it doesn't need to check.
It would be advisable to maintain a dictionary-esque data structure to keep track of certain aspects like that
and scaling to larger data sets would become much better.
"""

# is_k_anonymous(1, [], adult_anon)

"\nThe scaling behavior would not go well. It will always loop and check everything, even if it doesn't need to check.\nIt would be advisable to maintain a dictionary-esque data structure to keep track of certain aspects like that\nand scaling to larger data sets would become much better.\n"

### Question 4 

(15 points)

Write code to answer the query: "how many participants have never been married?"

*Hint*: filter the `adult_data` dataframe to contain only participants who were never married, then return the 0th element of the `shape` of the filtered dataframe.

*Hint*: if you have not used Pandas before, [this](https://pandas.pydata.org/pandas-docs/stable/comparison_with_sql.html) comparison with SQL might be useful.

In [68]:
query1 = adult_data[adult_data['Martial Status'] == 'Never-married']
query1.shape[0]

# 10683

10683

In [ ]:
# ignore this -- unsure how to delete empty rows

### Question 5 

(5 points)

In 2-5 sentences, answer the following:
- What is the sensitivity of `query1`, and why?

In [66]:
# Write your answer to Question 11 here
"""
The sensitivity of `query1` is 1. According to the course notes, counting queries always has a sensitivity of 1.
If the query is going through and counting rows, and then one row is changed, the query's output would
change by no more than one.
"""

"\nThe sensitivity of `query1` is 1. According to the course notes, counting queries always has a sensitivity of 1.\nIf the query is going through and counting rows, and then one row is changed, the query's output would\nchange by no more than one.\n"

### Question 6 

(15 points)

Define a function `laplace_mech` that implements the Laplace Mechanism for a query result consisting of a single real number. Your implementation should work for queries of any sensitivity (by passing sensitivity as an argument) and any value of `epsilon`.

*Hint*: use `np.random.laplace`.

In [23]:
def laplace_mech(query, sensitivity, epsilon):
    return query + np.random.laplace(loc=0, scale=sensitivity/epsilon)

In [35]:
laplace_mech(48273, 1, 1)

# 48273.62614440791

48273.92518471405

### Question 7 

(10 points)

Use your implementation of `laplace_mech` to produce a differentially private answer to your query from the last question, with `epsilon = 0.1`.

In [77]:
laplace_mech(query1.shape[0], 1, 0.1)

# 10683.322660193706

10685.62835011275